In [ ]:
!pip install segmentation-models-pytorch

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

### Импорт необходимых библиотек:

In [1]:
from torchvision import transforms
import numpy as np
import torch
import nibabel as nib
from pathlib import Path
import random
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.utils.losses import DiceLoss
from segmentation_models_pytorch.utils.metrics import IoU
from torch.optim import Adam
from PIL import Image
from torchvision import transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau

ModuleNotFoundError: ignored

### Проверяем доступность GPU:

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
import os

# 返回根目录
os.chdir('/content/gdrive/MyDrive/kaggle/cancer-net-pca-data')
# 返回当前目录
current_directory = os.getcwd()
print(current_directory)
!ls


### Определяем пути к файлам изображений и масок:

In [ ]:
DATA_ROOT = '/content/gdrive/MyDrive/kaggle/cancer-net-pca-data'
img_path = DATA_ROOT + '/images/ProstateX-0000_ep2d_diff_tra_7.nii'
mask_path = DATA_ROOT + '/masks/ProstateX-0004-Finding1-ep2d_diff_tra_DYNDIST_MIX_ADC_ROI.nii'

### Определяем целевой размер изображений после его изменения:

In [ ]:
target_size = (256, 256)

### Функция для предобработки изображений:

In [ ]:
def custom_preprocess_input(x):
    x = x / 255.0  # нормализация значений пикселей
    return x

### Определение аугментации данных:

In [ ]:
augmentation_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
])

In [ ]:
class MyProstateDataset(Dataset):
    def __init__(self, image_files, mask_files, preprocessing=None):
        self.image_files = image_files
        self.mask_files = mask_files
        self.data_len = len(self.image_files)
        self.preprocessing = preprocessing

    def __getitem__(self, index):
        img_path = self.image_files[index]
        mask_path = self.mask_files[index]

        # загрузка 3D данных с учетом формата изображений в датасете
        img_data = nib.load(img_path).get_fdata()
        mask_data = nib.load(mask_path).get_fdata()

        # Извлечение 2D среза для обработки
        slice_number = 9  # Номер среза
        img_slice = img_data[:, :, slice_number]
        mask_slice = mask_data[:, :, slice_number]

        # Предобработка изображения и маски
        img_slice = img_slice.astype(np.uint8)

        img_slice = Image.fromarray(img_slice)
        img_slice = img_slice.resize(target_size, resample=Image.BILINEAR)
        img_slice = np.array(img_slice)

        mask_slice = mask_slice.astype(np.uint8)
        mask_slice = Image.fromarray(mask_slice)
        mask_slice = mask_slice.resize(target_size, resample=Image.NEAREST)
        mask_slice = np.array(mask_slice)

        if self.preprocessing:
            img_slice = self.preprocessing(img_slice)
            img_slice = torch.as_tensor(img_slice)
        else:
            img_slice = img_slice / 255.0

        img_slice = img_slice.unsqueeze(0)  # Добавление размерности канала для изображения в градации серого
        mask_slice = torch.as_tensor(mask_slice, dtype=torch.uint8)

        return img_slice.float(), mask_slice

    def __len__(self):
        return self.data_len

### Создание списков файлов с изображениями и масками:

In [ ]:
image_files = sorted(list(Path(DATA_ROOT + '/images').rglob('*.nii')))
mask_files = sorted(list(Path(DATA_ROOT + '/masks').rglob('*.nii')))
print(image_files)

[PosixPath('/content/gdrive/MyDrive/kaggle/cancer-net-pca-data/images/ProstateX-0000_ep2d_diff_tra_7.nii'), PosixPath('/content/gdrive/MyDrive/kaggle/cancer-net-pca-data/images/ProstateX-0001_ep2d_diff_tra_8.nii'), PosixPath('/content/gdrive/MyDrive/kaggle/cancer-net-pca-data/images/ProstateX-0002_ep2d_diff_tra_7.nii'), PosixPath('/content/gdrive/MyDrive/kaggle/cancer-net-pca-data/images/ProstateX-0003_ep2d_diff_tra_6.nii'), PosixPath('/content/gdrive/MyDrive/kaggle/cancer-net-pca-data/images/ProstateX-0004_ep2d_diff_tra_7.nii'), PosixPath('/content/gdrive/MyDrive/kaggle/cancer-net-pca-data/images/ProstateX-0005_ep2d_diff_tra_7.nii'), PosixPath('/content/gdrive/MyDrive/kaggle/cancer-net-pca-data/images/ProstateX-0006_ep2d_diff_tra_7.nii'), PosixPath('/content/gdrive/MyDrive/kaggle/cancer-net-pca-data/images/ProstateX-0007_ep2d_diff_tra_8.nii'), PosixPath('/content/gdrive/MyDrive/kaggle/cancer-net-pca-data/images/ProstateX-0008_ep2d_diff_tra_8.nii'), PosixPath('/content/gdrive/MyDrive/k

In [ ]:
random.seed(42)
min_length = min(len(image_files), len(mask_files))
mask_files = random.sample(mask_files, min_length)
print(len(image_files))

200


### Разделим данные на обучающий и тестовый наборы:

In [ ]:
image_train, image_test, mask_train, mask_test = train_test_split(image_files, mask_files, test_size=0.3, random_state=42)

### Создадим объекты Dataset для обучения и тестирования:

In [ ]:
train_dataset = MyProstateDataset(image_train, mask_train, preprocessing=custom_preprocess_input)
test_dataset = MyProstateDataset(image_test, mask_test, preprocessing=custom_preprocess_input)

### Создадим Data_loader:

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False, num_workers=2)

### Определим архитектуру модели (PSPnet:

In [ ]:
BACKBONE = 'resnet34'
model = smp.PSPNet(BACKBONE, in_channels=1, classes=1, activation='sigmoid')

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 224MB/s]


In [ ]:
# Функции потерь и метрик
criterion = DiceLoss()
metrics = [IoU()]

In [ ]:
# Оптимизатор Adam
optimizer = Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)

In [ ]:
max_score = 0

model.to(device)

## **"Scheduler"** - это объект, представляющий планировщик изменения скорости обучения в процессе обучения нейронной сети.

* mode='min': планировщик будет уменьшать скорость обучения, если значение функции потерь на валидации уменьшается. Если бы было 'max', то он уменьшал бы скорость обучения, если значение метрики на валидации увеличивается.
*
* factor=0.5: Скорость обучения уменьшается в два раза при каждом уменьшении.
*
* patience=25: Если значение функции потерь на валидации не улучшается в течение 5 эпох, скорость обучения уменьшается.

In [ ]:
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=25, verbose=True)

### Обучение модели:

In [ ]:
best_valid_loss = float('inf')  # Начальное значение
epochs_without_improvement = 0
patience = 100 #общее количество, если модель не остановится

In [ ]:
train_losses = []
valid_losses = []

for epoch in range(100):
    print(f'Epoch: {epoch + 1}')

    train_logs = {'dice_loss': 0.0, 'iou_score': 0.0}
    valid_logs = {'dice_loss': 0.0, 'iou_score': 0.0}
    train_losses.append(train_logs['dice_loss'])
    valid_losses.append(valid_logs['dice_loss'])

    # Обучение
    model.train()


    for batch in train_loader:
        images, masks = batch
        optimizer.zero_grad()
        images, masks = images.to(device), masks.to(device)
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        train_logs['dice_loss'] += loss.item()

        for metric in metrics:
            train_logs['iou_score'] += metric(outputs, masks)

    train_logs['dice_loss'] /= len(train_loader)
    train_logs['iou_score'] /= len(train_loader)

    print(f'Train - Dice Loss: {train_logs["dice_loss"]:.4f}, IoU Score: {train_logs["iou_score"]:.4f}')

    # Валидация
    model.eval()


    with torch.no_grad():
        for batch in test_loader:
            images, masks = batch
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)
            loss = criterion(outputs, masks)
            valid_logs['dice_loss'] += loss.item()

            for metric in metrics:
                valid_logs['iou_score'] += metric(outputs, masks)

    valid_logs['dice_loss'] /= len(test_loader)
    valid_logs['iou_score'] /= len(test_loader)

    print(f'Validation - Dice Loss: {valid_logs["dice_loss"]:.4f}, IoU Score: {valid_logs["iou_score"]:.4f}')

    # Проверка, улучшилась ли функция потерь на валидации
    if valid_logs["dice_loss"] < best_valid_loss:
        best_valid_loss = valid_logs["dice_loss"]
        epochs_without_improvement = 0
        # Сохранение лучшей версии модели
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        epochs_without_improvement += 1

    # Проверка, достигнут ли критерий ранней остановы
    if epochs_without_improvement >= patience:
        print(f'Early stopping after {patience} epochs without improvement.')
        break

1. Dice Loss - это метрика, предназначенная для измерения сходства между предсказанными и истинными масками сегментации на изображениях. Dice Loss является функцией потерь (loss function) во время обучения модели сегментации. Его цель - максимизировать сходство между предсказанными и истинными масками, что приводит к лучшему качеству сегментации.
2. IoU Score (Индекс Жаккара) - это другая метрика, которая измеряет сходство между предсказанными и истинными масками сегментации. IoU Score также является популярной метрикой качества сегментации. Он обычно используется вместе с Dice Loss, чтобы оценить, насколько хорошо модель выполнила сегментацию. Важно максимизировать эту метрику для достижения лучших результатов.
Путем минимизации Dice Loss и максимизации IoU Score можно добиться более точной и совпадающей с реальностью сегментации объектов на изображениях.

По мере увеличения эпох значение Dice Loss на тренировочных данных снижается, что говорит об уменьшении потерь между предсказанными и истинными масками на тренировочных изображениях.
Но потери на валидации начинают увеличиваться после 5-й эпохи. Это может быть признаком переобучения модели.

Значение IoU Score на тренировочных данных увеличивается с количеством эпох. Но на валидации IoU Score начинает снижаться после 5-й эпохи, что также может указывать на переобучение.

## Визуализация  результатов:

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(range(1, len(train_losses) + 1), train_losses, label='Train Loss', marker='o')
plt.plot(range(1, len(valid_losses) + 1), valid_losses, label='Validation Loss', marker='o')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid()
plt.show()

In [ ]:
def visualize_predictions(model, data_loader, num_samples=10):
    model.eval()
    with torch.no_grad():
        for i, (images, masks) in enumerate(data_loader):
            if i >= num_samples:
                break
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)


            images = images.squeeze(1).cpu().numpy()
            masks = masks.cpu().numpy()
            predictions = outputs.cpu().numpy()

            for j in range(images.shape[0]):
                plt.figure(figsize=(12, 4))
                plt.subplot(1, 3, 1)
                plt.title("Original Image")
                plt.imshow(images[j], cmap='gray')

                plt.subplot(1, 3, 2)
                plt.title("Ground Truth Mask")
                plt.imshow(masks[j], cmap='viridis')

                plt.subplot(1, 3, 3)
                plt.title("Predicted Mask")
                plt.imshow(predictions[j, 0], cmap='viridis')
                plt.show()


visualize_predictions(model, test_loader, num_samples=10)